## Реализация алгоритма шифрования RSA

### Алгоритм генерации ключей

1. Пользователь A генерирует два больших простых числа p и q, отличных друг от друга. При этом |p-q| – большое число, хотя p и q имеют приблизительно одинаковый битовый размер.

In [1]:
from sympy import randprime

def gen_simple_numbers(bits):
    """
    Функция для генерации простых чисел.
    При этом |p-q| – большое число, хотя p и q имеют приблизительно одинаковый битовый размер.
    """
    while True:
        p = randprime(2 ** (bits - 1), 2**bits - 1)
        q = randprime(2 ** (bits - 1), 2**bits - 1)
        if p != q and abs(p - q) > 2 ** (bits // 2):
            return p, q

2. Держа p и q в секрете, Пользователь A вычисляет их произведение n=pq, которое называют модулем алгоритма.

In [2]:
def algorithm_module(p, q):
    """
    Функция для вычисления модуля алгоритма.
    """
    return p * q

3. Пользователь A вычисляет значение функции Эйлера для n по формуле φ(n)=(p-1)(q-1).

In [3]:
def euler_function(p, q):
    """
    Функция для вычисления значения функции Эйлера.
    """
    return (p - 1) * (q - 1)

4. Пользователь A выбирает целое число e, взаимно простое со значением функции φ(n). Это число называется экспонентой зашифрования.

In [4]:
from secrets import randbits
from math import gcd

def exponent_encryption(phi):
    """
    Функция для вычисления значения экспоненты зашифрования.
    """
    while True:
        e = randbits(16)
        if gcd(e, phi) == 1:
            return e

5. Пользователь A применяет расширенный алгоритм Евклида к паре чисел e и φ(n) и вычисляет значение d, удовлетворяющее соотношению ed≡1 mod φ(n). Это значение называется экспонентой расшифрования.

In [5]:
def gen_private_key(e, phi):
    """
    Функция для вычисления значения закрытого ключа.
    """
    return pow(e, -1, phi)

6. Пара (e,n) публикуется в качестве открытого ключа пользователя A, d является закрытым ключом и держится в секрете.

### Алгоритм зашифрования

In [6]:
from base64 import b64encode, b64decode

def rsa_encryption(message, e, n):
    """
    Функция для зашифрования сообщения с использованием алгоритма RSA.
    """
    bytes_data_encoded = b64encode(message.encode("utf-8"))
    block_size = (n.bit_length() - 1) // 8  # Размер блока
    blocks_encoded = [
        bytes_data_encoded[i : i + block_size]
        for i in range(0, len(bytes_data_encoded), block_size)
    ]

    for i in range(len(blocks_encoded)):
        blocks_encoded[i] = int.from_bytes(blocks_encoded[i], byteorder="big")

    blocks_encrypted = [pow(block, e, n) for block in blocks_encoded]

    return blocks_encrypted

### Алгоритм расшифрования

In [7]:
def rsa_decryption(blocks_encrypted, n, d):
    """
    Функция для расшифровки сообщения с использованием алгоритма RSA.
    """
    blocks_decrypted = [pow(block, d, n) for block in blocks_encrypted]
    blocks_decoded = [
        block.to_bytes((block.bit_length() + 7) // 8, byteorder="big")
        for block in blocks_decrypted
    ]
    reassembled = b"".join(blocks_decoded)
    message = b64decode(reassembled).decode("utf-8")
    return message

### Вывод результатов

In [8]:
message_original = "HELLO"
# message_original = input("Введите сообщение: ")

p, q = gen_simple_numbers(1024)
n = algorithm_module(p, q)
phi = euler_function(p, q)
e = exponent_encryption(phi)
d = gen_private_key(e, phi)
blocks_encrypted = rsa_encryption(message_original, e, n)
message_decrypted = rsa_decryption(blocks_encrypted, n, d)

# Генерация ключевых пар
# Шифрование текста
# Расшифрование текста

print("")
print(f"Открытый ключ:\ne = {e}\nn = {n}\n")
print(f"Закрытый ключ:\nd = {d}\n")
print(f"Зашифрованное сообщение:\n{blocks_encrypted}\n")
print(f"Расшифрованное сообщение:\n{message_decrypted}")


Открытый ключ:
e = 13379
n = 25287962768236181766834785377338875135944480016901063117346915095432207869742289331616940647370347108601057092053405665471138967798235681501624002217317159789661470195431613608735209881022622439775107914707798233304398103459326479493202418986186112462990663609990994228866273564832499133832430283894110431072507984743513820914375547799547748093584123764751369679392031535768298326908682168735173578643410112800104919717938822441959994660888045721659307737956963843929118287414521021800336778870133680102624298558737971482291863151869369737629723096983032181712732519913039458926449336588887623533268558037775844276589

Закрытый ключ:
d = 16287194496889990155080001913187015998687015793828870684070436308942322685818768754805529378757028255834913593110411586767681028889857004820950297264864486576538821187983721837691255216740558903022804761270431076790791423687048080857532345048506295769010430400425472551770736176706902237553931665768409416589360300236922542137534292